**META Learning**

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import drive

drive.mount('/gdrive')
%cd /gdrive/MyDrive/rdc/tei

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/rdc/tei


In [ ]:
import pandas as pd

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')


In [ ]:
train_df = train_df.sample(frac=0.1, random_state=42)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in train_df.columns:
    if col != 'id':
        train_df[col] = le.fit_transform(train_df[col])

for col in test_df.columns:
    if col != 'id':
        test_df[col] = le.fit_transform(test_df[col])
    if col == 'class':
        le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
        print(le_name_mapping)

In [ ]:
X = train_df.drop(['id', 'class'], axis=1)
y = train_df['class']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

Usando os melhores parâmetros para o XGBooster

In [ ]:
import xgboost as xgb

best_params = {
    'max_depth': 9,
    'subsample': 0.9276592101046933,
    'colsample_bytree': 0.9119456082162385,
    'learning_rate': 0.1,
    'n_estimators': 100,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'use_label_encoder': False,
    'random_state': 42
}

base_models = [
    ('xgb', xgb.XGBClassifier(**best_params)),
    ('dt', DecisionTreeClassifier(max_depth=4, random_state=42))
]

In [ ]:
meta_model = LogisticRegression()

Combinando dois modelos baseados em árvore

In [ ]:
stacking_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=10
)

In [ ]:
stacking_model.fit(X_train, y_train)
test_predictions = stacking_model.predict(test_df.drop('id', axis=1))

test_df['class'] = test_predictions
test_df['class'].replace({0: 'e', 1: 'p'}, inplace=True)
test_df[["id","class"]].to_csv("meta.csv", index=False)

Score Kaggle: 0.12964